In [7]:
import json
import cv2
import numpy as np

from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self):
        self.data = []
        with open('C:/Users/nadyl/Desktop/kursovaya/models/test_controlnet/fill50k/prompt.json', 'rt') as f:
            for line in f:
                self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        source_filename = item['source']
        target_filename = item['target']
        prompt = item['prompt']

        source = cv2.imread('C:/Users/nadyl/Desktop/kursovaya/models/test_controlnet/fill50k/' + source_filename)
        target = cv2.imread('C:/Users/nadyl/Desktop/kursovaya/models/test_controlnet/fill50k/' + target_filename)

        # Do not forget that OpenCV read images in BGR order.
        source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
        target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)

        # Normalize source images to [0, 1].
        source = source.astype(np.float32) / 255.0

        # Normalize target images to [-1, 1].
        target = (target.astype(np.float32) / 127.5) - 1.0

        return dict(jpg=target, txt=prompt, hint=source)


In [8]:
dataset = MyDataset()
print(len(dataset))

item = dataset[1234]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)


50000
burly wood circle with orange background
(512, 512, 3)
(512, 512, 3)


In [7]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict


# Configs
resume_path = 'C:/Users/nadyl/Desktop/kursovaya/models/test_controlnet/models/v2-1_512-ema-pruned.ckpt'
batch_size = 2
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False


# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('C:/Users/nadyl/Desktop/kursovaya/models/test_controlnet/models/cldm_v21.yaml').cpu()
model.load_state_dict(load_state_dict(resume_path, location='cuda'), strict=False)
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control


# Misc
dataset = MyDataset()
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(gpus=1, precision=32, callbacks=[logger])


# Train!
trainer.fit(model, dataloader)


ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 222.98 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [C:/Users/nadyl/Desktop/kursovaya/models/test_controlnet/models/cldm_v21.yaml]
Loaded state_dict from [C:/Users/nadyl/Desktop/kursovaya/models/test_controlnet/models/v2-1_512-ema-pruned.ckpt]


RuntimeError: Error(s) in loading state_dict for ControlLDM:
	size mismatch for model.diffusion_model.time_embed.0.weight: copying a param with shape torch.Size([1280, 320]) from checkpoint, the shape in current model is torch.Size([640, 160]).
	size mismatch for model.diffusion_model.time_embed.0.bias: copying a param with shape torch.Size([1280]) from checkpoint, the shape in current model is torch.Size([640]).
	size mismatch for model.diffusion_model.time_embed.2.weight: copying a param with shape torch.Size([1280, 1280]) from checkpoint, the shape in current model is torch.Size([640, 640]).
	size mismatch for model.diffusion_model.time_embed.2.bias: copying a param with shape torch.Size([1280]) from checkpoint, the shape in current model is torch.Size([640]).
	size mismatch for model.diffusion_model.input_blocks.0.0.weight: copying a param with shape torch.Size([320, 4, 3, 3]) from checkpoint, the shape in current model is torch.Size([160, 4, 3, 3]).
	size mismatch for model.diffus

In [4]:
!pip install pytorch_lightning 